In [1]:
import sys
import numba
import swifter
import timeit
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

In [3]:
data = pd.read_csv('../Data/bid_fe.csv')
data.sample(5)

,ListingKey,BidCreationDate,MemberKey,Bid_Key,BidAmount,BidMinimumRate,BidMinimumYield,ListingStatus,Status,Member_Key,MemberCreationDate,MemberRoles,ListingCreationDate,ListingStartDate,Listing_Key,FundedOrNot,RepaidOrNot
6731080,6E6C3475865169598599922,2010-02-17 19:38:01,D6BB3391182847811E3DC85,54E23581153973883C284F3,25.0,NaN,NaN,Completed,Winning,D6BB3391182847811E3DC85,2007-06-14 14:29:00,"Lender,TRADER",2010-02-16 20:47:37,2010-02-17 16:06:13,6E6C3475865169598599922,True,True
9516913,2881353786426982309F801,2012-01-31 09:01:42,CBC2337413289524800EBCB,714536425270767271BCEAC,300.0,NaN,NaN,Cancelled,Winning,CBC2337413289524800EBCB,2006-11-07 18:30:00,"Borrower,Lender,TRADER",2012-01-31 04:20:30,2012-01-31 09:00:42,2881353786426982309F801,False,NaN
6157776,2DC3346056728787919417B,2009-09-03 09:40:05,26D4336515615999253C10E,4B753566751338604FDC8BB,0.0,0.0945,0.0845,Completed,Outbid,26D4336515615999253C10E,2006-04-10 17:19:00,"Borrower,Lender,Group Leader,TRADER",2009-08-24 06:08:16,2009-08-24 15:53:34,2DC3346056728787919417B,True,True
5048730,8C94342483596710973D6DC,2008-06-24 06:41:19,0E9F3415250765736F40DE9,569A352825258179732017E,50.0,NaN,NaN,Completed,Winning,0E9F3415250765736F40DE9,2008-03-03 05:24:06,Lender,2008-06-20 19:51:18,2008-06-20 19:54:05,8C94342483596710973D6DC,True,False
6276013,E37034573595920626F266C,2009-07-27 15:52:16,55E73382781601133B7618D,F5CE3562824985082183762,46.0,NaN,NaN,Completed,Winning,55E73382781601133B7618D,2007-02-19 07:28:00,Lender,2009-07-21 20:37:20,2009-07-22 15:55:02,E37034573595920626F266C,True,NaN


In [4]:
len(data)

9733452

In [5]:
hashmap = dict()
memberkeys = sorted(list(set(data.MemberKey)))[60000:]
len(memberkeys)

5756

In [5]:
start = timeit.default_timer()
for key in memberkeys:
    hashmap[key] = data.query("MemberKey == '{0}'".format(key))
stop = timeit.default_timer()
print "Hashmap generated in {0} seconds".format((stop-start)) 

Hashmap generated in 1756.6564536 seconds


In [6]:
data = data[data['MemberKey'].isin(memberkeys)]

In [7]:
len(data)

863537

## Lender Winning Bids

In [8]:
def lender_winning_bids(x):
    return hashmap[x["MemberKey"]].query("Status == 'Winning' and ListingStartDate < '{0}'".format(x["ListingStartDate"]))["Bid_Key"].nunique()

start = timeit.default_timer()    
data["LenderWinningBids"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_winning_bids, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 
# data[["MemberKey", "ListingKey", "ListingStartDate", "Bid_Key", "BidCreationDate", "Status", "LenderWinningBids"]].query("MemberKey=='365434772269169153CB3D3'").sort_values("ListingStartDate")

Feature engineering completed in 2318.6488344 seconds


## Lender Total Bids

In [9]:
def lender_total_bids(x):
    return hashmap[x["MemberKey"]].query("ListingStartDate < '{0}'".format(x["ListingStartDate"]))["Bid_Key"].nunique()

start = timeit.default_timer()    
data["LenderTotalBids"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_total_bids, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 
# data[["MemberKey", "ListingKey", "ListingStartDate", "Bid_Key", "BidCreationDate", "Status", "LenderWinningBids", "LenderTotalBids"]].head()

Feature engineering completed in 1999.4617398 seconds


In [12]:
data[["MemberKey", "ListingKey", "ListingStartDate", "Bid_Key", "BidCreationDate", "Status", "LenderWinningBids", "LenderTotalBids"]].query("MemberKey=='{0}'".format(memberkeys[0])).sort_values("ListingStartDate")

,MemberKey,ListingKey,ListingStartDate,Bid_Key,BidCreationDate,Status,LenderWinningBids,LenderTotalBids
617824,E9483375866518194004048,2F2833760777297309E8E1C,2006-12-18 13:50:43,107633774655206954E8D4B,2006-12-21 15:18:50,Outbid,0,0
622353,E9483375866518194004048,E17233764074001975E464B,2006-12-19 11:15:29,31413375814710850E21971,2006-12-21 15:21:07,Outbid,0,1
625021,E9483375866518194004048,A12C33757831886428C676D,2006-12-19 18:45:02,83E433772460946946F8563,2006-12-21 15:32:57,Winning,0,2
626730,E9483375866518194004048,6F8F33774319887253D274C,2006-12-20 03:35:43,82AE3375913833507B07486,2006-12-21 20:05:00,Winning,1,3
669774,E9483375866518194004048,B8643378137374332EFEE76,2006-12-29 12:00:44,2ECB3376937990631B3E1EC,2006-12-29 16:35:07,Winning,2,4
692066,E9483375866518194004048,E91A33771657539829E68EB,2007-01-03 11:39:40,5A8D337756952117220A71B,2007-01-04 15:01:16,Winning,3,5
710501,E9483375866518194004048,92CA337765305664138DEEB,2007-01-05 19:55:40,4C193379560918595D9AD91,2007-01-12 13:12:31,Outbid,4,6
752003,E9483375866518194004048,103833784659606072D276E,2007-01-13 04:56:43,A38533795113576561EA120,2007-01-17 01:29:23,Outbid,4,7
758675,E9483375866518194004048,01C733786269340506C9D0F,2007-01-14 08:42:21,7A5B3378125127481BA2B58,2007-01-14 20:14:31,Outbid,4,8
757399,E9483375866518194004048,D2A2337775624004187DFE6,2007-01-14 10:05:30,6F2A3379390032462663DBD,2007-01-19 07:47:37,Outbid,4,9


## Save Data

In [11]:
data.to_csv("lender_bid_fe_65k.csv", index=False)